## Exercise - DL Tutorial 08

### Student names:  Franz Schulze, Benedikt Bauer, David Heim

Submit you solution by 30 June 23:59 to manuel.milling@informatik.uni-augsburg.de or maurice.gerczuk@informatik.uni-augsburg.de

In [16]:
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Embedding, Dropout


w2v_embedding_file = "./res/data/embeddings/word2vec-40k-wiki-news-300d.vec"#"./res/data/embeddings/word2vec-40k-wiki-news-300d.vec"
ewe_embedding_file = "./res/data/embeddings/ewe-40k-wiki-news-300d.vec"

train_tsv = "./res/data/isear/train.tsv "
val_tsv = "./res/data/isear/val.tsv"
test_tsv = "./res/data/isear/test.tsv"

oov_id=1
pad_id=0
seq_length=128
batch_size=32
epochs=5

lr=0.005

2. Load word2vec embedding matrix and create word-index-dictionary.

In [17]:
def read_embedding_matrix(embedding_file):
    words = []
    embeddings = []
    with open(embedding_file, encoding='utf-8') as f:
        for line in f.readlines():
            line = line.replace("\n", "")
            col = line.split(" ")
            words.append(col[0])
            embeddings.append(np.array([float(c) for c in col[1:]]))
    embedding_matrix = np.row_stack(embeddings)
    pad_embedding = np.zeros(embedding_matrix.shape[1])
    oov_embedding = np.expand_dims(np.mean(embedding_matrix, axis=0), 0)
    embedding_matrix = np.row_stack([pad_embedding, embedding_matrix, oov_embedding])
    w2v_word2idx = {word: i for i, word in enumerate(words)}
    return embedding_matrix, w2v_word2idx

w2v_emb_matrix, w2v_word2idx = read_embedding_matrix(w2v_embedding_file)
print(f"w2v_emb_matrix:\t\t{w2v_emb_matrix.shape}")
print(f"w2v_word2idx shape:\t{len(w2v_word2idx)}")
# num_words = 40000
# embedding_dim = 300
# def read_embedding_matrix(embedding_file):
#     file = open(embedding_file, 'r', encoding='utf-8')
#     w2v_emb_matrix = np.array(file,(num_words + 2, embedding_dim))
#     w2v_emb_matrix[0] = np.zeros(embedding_dim)
#     w2v_emb_matrix[1] = np.mean(embedding_dim, axis=1)
#     dict_word2idx = {word: i + 2 for i, word in enumerate(w2v_emb_matrix[:, 0])}
#     # words = []
#     # embeddings = []
#     # with open(w2v_embedding_file) as file:
#     #     for line in file.readlines():
#     #         line = line.replace("\n","")
#     #         columns = line.split(" ")
#     #         words.append(columns[0])
#     #         embeddings.append(np.array([float(f) for f in columns[1:]]))

#     # emb_matrix = np.row_stack(embeddings)
#     # # Put pad embedding at position 0.
#     # pad_emb = np.zeros((1, emb_matrix.shape[1]))
#     # # Put oov embedding at position 1.
#     # oov_emb = np.expand_dims(np.mean(emb_matrix, axis=0), 0)
#     # emb_matrix = np.row_stack([pad_emb, oov_emb, emb_matrix])
    
#     # # Caution: added pad and oov token.
#     # w2v_word2idx = {word: i + 2 for i, word in enumerate(words)}
#     # return emb_matrix, w2v_word2idx

# w2v_emb_matrix, w2v_word2idx = read_embedding_matrix(w2v_embedding_file)
# print(f"w2v_emb_matrix:\t\t{w2v_emb_matrix.shape}")
# print(f"w2v_word2idx shape:\t{len(w2v_word2idx)}")

w2v_emb_matrix:		(40002, 300)
w2v_word2idx shape:	40000


3. Prepare data:
- Load the sentences from the tsv files.
- Unify sentences (lower case, remove punctuation, etc.).
- Split sentences into words.
- Cut and zero pad sentences.
- Map words to indices.
- Map string labels to integers.

In [18]:
def read_tsv(tsv, word2idx, oov_id=1, pad_id=0, seq_length=128):
    texts = []
    labels = []
    with open(tsv, encoding='utf-8') as f:
        for line in f.readlines():
            line = line.replace("\n", "")
            line_splitted = line.split("\t")
            text = line_splitted[0]
            label = line_splitted[1]
            text = re.sub(r'[^\w\s]', '', text)
            text = re.sub(r'\s\s+', ' ', text)
            text = text.strip()
            text_list = text.lower().split(" ")
            text_to_words = [t for t in text_list if len(t) > 0]
            text_to_words = text_to_words[:seq_length]
            text_to_words = [word2idx.get(token, oov_id) for token in text_to_words]
            text_to_words = text_to_words + (seq_length - len(text_to_words)) * [pad_id]
            texts.append(text_to_words)
            labels.append(label)
    return np.array(texts), np.array(labels)
train_X, train_y = read_tsv(train_tsv, w2v_word2idx, oov_id=oov_id, pad_id=pad_id, seq_length=seq_length)
val_X, val_y = read_tsv(val_tsv, w2v_word2idx, oov_id=oov_id, pad_id=pad_id, seq_length=seq_length)
test_X, test_y = read_tsv(test_tsv, w2v_word2idx, oov_id=oov_id, pad_id=pad_id, seq_length=seq_length)

# convert labels from string to int

print(f"x_train shape:\t\t{train_X.shape}")
print(f"x_validation shape:\t{val_X.shape}")
print(f"x_test shape:\t\t{test_X.shape}")
print(f"y_train shape:\t\t{train_y.shape}")
print(f"y_validation shape:\t{val_y.shape}")
print(f"y_test shape:\t\t{test_y.shape}")

x_train shape:		(5976, 128)
x_validation shape:	(752, 128)
x_test shape:		(736, 128)
y_train shape:		(5976,)
y_validation shape:	(752,)
y_test shape:		(736,)


4. Initialise, train  and evaluate model.

In [19]:
model = keras.Sequential([
    Embedding(input_dim=len(w2v_word2idx.keys()) + 2, output_dim=300, mask_zero=True, 
              embeddings_initializer=keras.initializers.Constant(w2v_emb_matrix), trainable=False),
    Bidirectional(LSTM(64, return_sequences=True, dropout=0.5)),
    Bidirectional(LSTM(64, dropout=0.5)),
    Dense(7, activation='softmax')]
)
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics='accuracy')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 300)         12000600  
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 128)         186880    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 903       
Total params: 12,287,199
Trainable params: 286,599
Non-trainable params: 12,000,600
_________________________________________________________________


In [20]:
history = model.fit(x=train_X,y=train_y, epochs=epochs, batch_size=batch_size, validation_data=(val_X, val_y))

Epoch 1/5


UnimplementedError:  Cast string to float is not supported
	 [[node sparse_categorical_crossentropy/Cast (defined at <ipython-input-20-4ccd1de56494>:1) ]] [Op:__inference_train_function_110853]

Function call stack:
train_function


In [ ]:
model.evaluate(x=test_X, y=test_y)

5. EWE embeddings.

In [ ]:
ewe_emb_matrix, ewe_word2idx = read_embedding_matrix(ewe_embedding_file)
train_X, train_y = read_tsv(train_tsv, ewe_word2idx, oov_id=oov_id, pad_id=pad_id, seq_length=seq_length)
val_X, val_y = read_tsv(val_tsv, ewe_word2idx, oov_id=oov_id, pad_id=pad_id, seq_length=seq_length)
test_X, test_y = read_tsv(test_tsv, ewe_word2idx, oov_id=oov_id, pad_id=pad_id, seq_length=seq_length)

# Convert labels from string to int.
encoder = LabelEncoder()
encoder.fit(train_y)

train_y = encoder.transform(train_y)
val_y = encoder.transform(val_y)
test_y = encoder.transform(test_y)

In [ ]:
model_ewe = keras.Sequential([
    Embedding(input_dim=len(ewe_word2idx.keys()) + 2, output_dim=300, mask_zero=True, 
              embeddings_initializer=keras.initializers.Constant(ewe_emb_matrix), 
                     trainable=False),
    Bidirectional(LSTM(64, return_sequences=True, dropout=0.5)),
    Bidirectional(LSTM(64, dropout = 0.5)),
    Dense(7, activation='softmax')]
)
model_ewe.compile(
    loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics='accuracy')
model_ewe.summary()
history = model_ewe.fit(x=train_X, y=train_y, epochs=5, batch_size=32, validation_data=(val_X, val_y))
model_ewe.evaluate(x=test_X, y=test_y)

6. Custom word embeddings.

In [ ]:
model_own = keras.Sequential([
    Embedding(input_dim=len(w2v_word2idx.keys()) + 2, output_dim=300, mask_zero=True, trainable=True),
    Bidirectional(LSTM(64, return_sequences=True, dropout=0.5)),
    Bidirectional(LSTM(64, dropout = 0.5)),
    Dense(7, activation='softmax')]
)
model_own.compile(
    loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics='accuracy')

In [ ]:
history = model_own.fit(x=train_X, y=train_y, epochs=5, batch_size=32, validation_data=(val_X, val_y))
model_own.evaluate(test_X, test_y)